In [36]:
import config.mongo_setup as mongo_setup
from models import Tweets, Replies
import pandas as pd
mongo_setup.global_init()

candidates = [('lhmandetta',2726301380),
('EduardoLeite_',123891446),
('DatenaOficial',68722955),
('SF_Moro',1113094855281000000),
('cirogomes',33374761),
('jdoriajr',64706049),
('LulaOficial',2670726740),
('jairbolsonaro',128372940),]

In [37]:

objs = Tweets.objects().all().values_list('id', 'tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')

corupus = pd.DataFrame.from_records(data=objs, columns=['id','tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
# corupus = corupus.drop(columns=['author_id','created_at','retweet_count','reply_count','like_count','quote_count','id'])
# corupus = corupus.rename(columns={"tweet_id": "id"})
corupus.head()

,id,tweet_id,author_id,created_at,text,lang,retweet_count,reply_count,like_count,quote_count
0,613ec5589eb2bd97646d86ad,1437158653623160837,2670726740,2021-09-12 20:58:18,@dilmabr 🙏🏼,und,123,76,6411,7
1,613ec5599eb2bd97646d86ae,1437054980884684804,2670726740,2021-09-12 14:06:21,Recomendação de leitura para o domingo: Amor à...,pt,1812,1529,18236,271
2,613ec5599eb2bd97646d86af,1436397090003881984,2670726740,2021-09-10 18:32:07,"“Olho pra trás, vejo a estrada que eu trilhei,...",pt,783,538,7294,84
3,613ec5599eb2bd97646d86b0,1436156966817181698,2670726740,2021-09-10 02:37:58,"@leandraleal Feliz Aniversário, companheira! 💫",pt,87,84,7371,7
4,613ec55a9eb2bd97646d86b1,1435999486162186244,2670726740,2021-09-09 16:12:11,@MussumAlive @manobrown 👊🏼,und,10,15,636,1


In [38]:
corupus = corupus.drop(columns=['author_id','created_at','retweet_count','reply_count','like_count','quote_count','id'])
corupus = corupus.rename(columns={"tweet_id": "id", 'lang':'language'})
corupus.head()

,id,text,language
0,1437158653623160837,@dilmabr 🙏🏼,und
1,1437054980884684804,Recomendação de leitura para o domingo: Amor à...,pt
2,1436397090003881984,"“Olho pra trás, vejo a estrada que eu trilhei,...",pt
3,1436156966817181698,"@leandraleal Feliz Aniversário, companheira! 💫",pt
4,1435999486162186244,@MussumAlive @manobrown 👊🏼,und


In [39]:
corupus.to_json('azure_formated.json', indent=4, orient="records")